# GAWS 明文实现

主要包括以下算法：

- SNP过滤
- PCA
- 优势比
- 趋势卡方计算（检验）

## 1. SNP过滤

## 2. PCA

## 3. 优势比

## 4. 趋势卡方计算（检验）

### 原理：

如下面$2\times3$的列联表。其中，$R_1=N_{11}+N_{12}+N_{13}$，$C_1=N_{11}+N_{21}$，etc.

|         |B=1      |B=2      |B=3      |sum      |
|:-------:|:-------:|:-------:|:-------:|:-------:|
|A=1      |$N_{11}$ |$N_{12}$ |$N_{13}$ |$R_1$    |
|A=2      |$N_{21}$ |$N_{22}$ |$N_{23}$ |$R_2$    |
|sum      |$C_1$    |$C_2$    |$C_3$    |$N$      |

首先计算：①$T=\sum_{i=1}^k t_{i}(N_{1i}R_2 - N_{2i}R_1)$

然后计算②方差: $Var(T) = \frac{R_1 R_2}{N}(\sum_{i=1}^{k} t_i^2 C_i(N-C_i) - 2\sum_{i=1}^{k-1}\sum_{j=i+1}^{k}t_i t_j C_i C_j)$

最后计算: ③  $\frac{T}{\sqrt{Var(T)}}$

注：对于上面权重 $t$ 的选择，一般选择$(1, 1, 0)$ / $(0, 1, 1)$ / $(0, 1, 2)$

### 例子:

假设有如下列联表：

|         | Genotype aa | Genotype Aa | Genotype AA | Sum |
|:-------:|:-----------:|:-----------:|:-----------:|:---:|
|Controls | 20          | 20          | 20          |60   |
|Cases    | 10          | 20          | 30          |60   |
|Sum      | 30          | 40          | 50          |120  |
